<a href="https://colab.research.google.com/github/Regret49/KSU-Neural-Network/blob/main/Dobo_Scott_Original_Code_3D_to_Video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Written on 4/18/2022 for Scott Dobo's KSU Internship

https://github.com/cvdfoundation/kinetics-dataset

---

https://github.com/RaivoKoot/Video-Dataset-Loading-Pytorch

# Import statements

In [ ]:
!pip install torchdiffeq

In [ ]:
#The tools to look at and unpack the Kinectis400 dataset is gotten here
!git clone https://github.com/cvdfoundation/kinetics-dataset.git
!cd kinetics-dataset

Cloning into 'kinetics-dataset'...
remote: Enumerating objects: 130, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 130 (delta 33), reused 20 (delta 19), pack-reused 82
Receiving objects: 100% (130/130), 32.22 KiB | 8.05 MiB/s, done.
Resolving deltas: 100% (47/47), done.


In [ ]:
#the tools used to look at and analyze the dataset are from her
!git clone https://github.com/RaivoKoot/Video-Dataset-Loading-Pytorch.git
!cd Video-Dataset-Loading-Pytorch

Cloning into 'Video-Dataset-Loading-Pytorch'...
remote: Enumerating objects: 408, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 408 (delta 70), reused 54 (delta 54), pack-reused 328
Receiving objects: 100% (408/408), 6.69 MiB | 11.88 MiB/s, done.
Resolving deltas: 100% (192/192), done.


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import shutil

from torchdiffeq import odeint_adjoint as odeint

import matplotlib.pyplot as plt 
import numpy as np
import time

# Code to set up video dataset

In [ ]:
#downloads part 0 of the kinetics 400 dataset as it is about 1000 videos and anymore seemed like overkill
#the third is a CSV that contains every video in the training datset and what action is has
!wget https://s3.amazonaws.com/kinetics/400/train/part_0.tar.gz
!wget https://s3.amazonaws.com/kinetics/400/train/k400_train_path.txt
!wget https://s3.amazonaws.com/kinetics/400/annotations/train.csv

--2022-05-16 01:36:29--  https://s3.amazonaws.com/kinetics/400/train/part_0.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.141.208
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.141.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1631315939 (1.5G) [application/x-tar]
Saving to: ‘part_0.tar.gz’

part_0.tar.gz        32%[=====>              ] 512.00M  45.3MB/s    eta 24s    

In [ ]:
#makes directorys that are later used
!mkdir k400
!mkdir k400_targz
!mkdir /content/k400_targz/train
!mkdir rgb

In [ ]:
#copies failes to make them more accessible
shutil.copyfile('/content/part_0.tar.gz', '/content/k400_targz/train/part_0.tar.gz')
shutil.copyfile('/content/k400_train_path.txt', '/content/k400_targz/train/k400_train_path.txt')

In [ ]:
#this runs the extractor so that the 1000 videos are loaded properly to be used rather than compressed
!bash /content/kinetics-dataset/k400_extractor.sh

In [ ]:
shutil.copyfile('/content/Video-Dataset-Loading-Pytorch/Kinetics400/videos_to_frames.py', '/content/k400/videos_to_frames.py')

'/content/k400/videos_to_frames.py'

Remeber to edit the file you just copied as this redefines what it looks at, but it could as easily be switched to any other folder, but this is what I used

From:

video_path = '/home/raivo/data1/kinetics/videos/all'

rgb_out_path = 'rgb'

To:

video_path = '/content/k400/train'

rgb_out_path = '/content/rgb'

In [ ]:
#if edited properly should show 985 videos to convert and take about 20 minutes to run
#this takes all of the videos and seperates into their frames and then formats for other code to use
!python3 /content/k400/videos_to_frames.py

waiting for all videos to be completed. 985 videos
This can take an hour or two depending on dataset size
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6792000] moov atom not found
VIDIOC_REQBUFS: Inappropriate ioctl for device
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x4118c00] moov atom not found
VIDIOC_REQBUFS: Inappropriate ioctl for device
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x6792c00] moov atom not found
VIDIOC_REQBUFS: Inappropriate ioctl for device
[mov,mp4,m4a,3gp,3g2,mj2 @ 0xbc42600] moov atom not found
VIDIOC_REQBUFS: Inappropriate ioctl for device
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x4a78a00] moov atom not found
VIDIOC_REQBUFS: Inappropriate ioctl for device
[mov,mp4,m4a,3gp,3g2,mj2 @ 0xb252400] moov atom not found
VIDIOC_REQBUFS: Inappropriate ioctl for device
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x3cfb600] moov atom not found
VIDIOC_REQBUFS: Inappropriate ioctl for device
all done


In [ ]:
#copies files so that they are where they need to be
shutil.copyfile('/content/Video-Dataset-Loading-Pytorch/Kinetics400/process_annotation_file.py', '/content/process_annotation_file.py')
shutil.copyfile('/content/Video-Dataset-Loading-Pytorch/Kinetics400/labels_to_id.csv', '/content/labels_to_id.csv')
shutil.copyfile('/content/Video-Dataset-Loading-Pytorch/video_dataset.py', '/content/video_dataset.py')

'/content/video_dataset.py'

In [ ]:
#this would need to be edited if the annotation file is renamed
#it takes that CSV and then tell the program what every folder is showing for the network
!python3 /content/process_annotation_file.py

In [ ]:
from video_dataset import  VideoFrameDataset, ImglistToTensor
from torchvision import transforms
import torch
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import os

In [ ]:
root = os.path.join(os.getcwd(), '/content/rgb')
annotation_file = os.path.join(root, '/content/train_processed.txt')

In [ ]:
preprocess = transforms.Compose([
        ImglistToTensor(),  # list of PIL images to (FRAMES x CHANNELS x HEIGHT x WIDTH) tensor
        transforms.Resize(299),  # image batch, resize smaller edge to 299
        transforms.CenterCrop(299),  # image batch, center crop to square 299x299
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

In [ ]:
#these currently go to the exact same path, but when changed they would need to be redefined
#the most imporant parts of this are num_segments=5 and frames_per_segment=1
#basically these cut the video into x equal segments and then take Y frame from each of those
#they determine what the network sees as it should ignore any file not picked
trainset = VideoFrameDataset(
        root_path=root,
        annotationfile_path=annotation_file,
        num_segments=5,
        frames_per_segment=1,
        imagefile_template = 'frame_{0:012d}.jpg',
        transform= preprocess,
        test_mode=False
)

testset = VideoFrameDataset(
        root_path=root,
        annotationfile_path=annotation_file,
        num_segments=5,
        frames_per_segment=1,
        imagefile_template = 'frame_{0:012d}.jpg',
        transform= preprocess,
        test_mode=False
)

# ODE Code

In [ ]:
#VideoFrameDataset was used to format this for the dataloader class to then accept it
trainloader = torch.utils.data.DataLoader(
        dataset=trainset,
        batch_size=12,
        shuffle=True,
        num_workers=2,
        pin_memory=True
    )
testloader = torch.utils.data.DataLoader(
        dataset=testset,
        batch_size=12,
        shuffle=True,
        num_workers=2,
        pin_memory=True
    )

In [ ]:
class MyNet(nn.Module):
	def __init__(self, path):
		super(MyNet, self).__init__()
		self.path = path

	def num_params(self):
		return sum(p.numel() for p in self.parameters() if p.requires_grad)

	#if moved out of collab and saved locally this line would allow the network to call a previous so training does not need to be repeated
	def load(self):
		self.load_state_dict(torch.load('./' + self.path + '.pth'))

In [ ]:
class ODEFunc(nn.Module):
	def __init__(self, dim):
		super(ODEFunc, self).__init__()
		self.gn = nn.GroupNorm(min(32, dim), dim)
		self.conv = nn.Conv2d(dim, dim, 3, padding = 1)
		self.nfe = 0 # time counter

	def forward(self, t, x):
		self.nfe += 1
		x = self.gn(x)
		x = F.relu(x)
		x = self.conv(x)
		x = self.gn(x)
		return x

In [ ]:
class ODEBlock(nn.Module):
	def __init__(self, odefunc):
		super(ODEBlock, self).__init__()
		self.odefunc = odefunc
		self.integration_time = torch.tensor([0, 1]).float()

	def forward(self, x):
		out = odeint(self.odefunc, x, self.integration_time, rtol=1e-1, atol=1e-1) # high tolerances for speed

		# first dimension is x(0) and second is x(1),
		# so we just want the second
		return out[1]

In [ ]:
class ODENet(MyNet):
	def __init__(self):
		super(ODENet, self).__init__('Video_odenet')
		self.conv1 = nn.Conv3d(5, 6, 3)
		self.gn = nn.GroupNorm(6, 6)
		self.odefunc = ODEFunc(6)
		self.odeblock = ODEBlock(self.odefunc)
		self.pool = nn.AvgPool2d(2)
		#the 500 here is the index size, so the higher the number the more memory used
		#at 500 the program runs fine, but when increased to 1000 it would crash
		#lower than 500 might cause the program to not function
		#was not enough time to properly test all values for the exact breakpoints
		self.fc = nn.Linear(6 * 148 * 148, 500)

	def forward(self, x):
		x = self.conv1(x)
		x = F.relu(self.gn(x))

		y = torch.squeeze(x)

		y = self.odeblock(y)

		y = self.pool(y)

		y = y.view(-1, 6 * 148 * 148)

		y = self.fc(y)

		return y

In [ ]:
def train(net):
	#orignal n = 60000 / (5*16) but reduced to make training much faster than otherwise would be
	#determines the size of each epoch so larger numbers take longer
	n = 120 / (5*12) # batch size 12
	criterion = nn.CrossEntropyLoss()
	optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
	
	#defines the number of epochs so higher number means more training
	for epoch in range(7):  
		running_loss = 0.0
		for i, data in enumerate(trainloader, 0):
			# get the inputs; data is a list of [inputs, labels]
			inputs, labels = data

			# zero the parameter gradients
			optimizer.zero_grad()

			# forward + backward + optimize
			outputs = net(inputs)
			loss = criterion(outputs, labels)
			loss.backward()
			optimizer.step()

			# print statistics
			running_loss += loss.item()
			if i % n == n-1:    
				print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / n))
				running_loss = 0.0

	print('Finished Training')
	torch.save(net.state_dict(), './' + net.path + '.pth')

In [ ]:
def test(net):
	initial_time = time.time()
	correct = 0
	total = 0
	with torch.no_grad():
		for data in testloader:
			images, labels = data
			batch_size = images.shape[0]
			outputs = net(images)
			_, predicted = torch.max(outputs.data, 1)
			total += labels.size(0)
			correct += (predicted == labels).sum().item()
	final_time = time.time()
	print('Accuracy of the ' + net.path + ' network on the test set: %.2f %%' % (100 * correct / total))
	print('Time: %.2f seconds' % (final_time - initial_time))
	return(100 * correct / total)

In [ ]:
odenet = ODENet()
train(odenet)

[1,     2] loss: 6.482
[1,     4] loss: 9.421
[1,     6] loss: 9.360
[1,     8] loss: 8.556
[1,    10] loss: 8.990
[1,    12] loss: 9.318
[1,    14] loss: 7.931
[1,    16] loss: 7.906
[1,    18] loss: 7.798
[1,    20] loss: 7.442
[1,    22] loss: 6.574
[1,    24] loss: 6.591
[1,    26] loss: 6.568
[1,    28] loss: 6.586
[1,    30] loss: 6.648
[1,    32] loss: 6.499
[1,    34] loss: 6.526
[1,    36] loss: 6.491
[1,    38] loss: 6.544
[1,    40] loss: 6.167
[1,    42] loss: 6.617
[1,    44] loss: 6.063
[1,    46] loss: 6.178
[1,    48] loss: 6.158
[1,    50] loss: 6.330
[1,    52] loss: 6.303
[1,    54] loss: 6.406
[1,    56] loss: 6.242
[1,    58] loss: 6.227
[1,    60] loss: 6.491
[1,    62] loss: 6.485
[1,    64] loss: 6.151
[1,    66] loss: 6.556
[1,    68] loss: 6.269
[1,    70] loss: 6.358
[1,    72] loss: 6.445
[1,    74] loss: 6.284
[1,    76] loss: 6.771
[1,    78] loss: 6.106
[1,    80] loss: 6.369
[1,    82] loss: 6.390
[2,     2] loss: 6.034
[2,     4] loss: 5.694
[2,     6] 

In [ ]:
odenet.load()
test(odenet)

#Test 1   1 Epoch 60/60
#Accuracy of the Video_odenet network on the test set: 14.30 %
#Time: 807.26 seconds

#Test 2   1 Epoch 60/60
#Accuracy of the Video_odenet network on the test set: 14.90 %
#Time: 798.89 seconds

#Test 3   5 Epochs 120/60
#Accuracy of the Video_odenet network on the test set: 84.59 %
#Time: 1182.26 seconds

#Test 4   5 Epochs 120/60
#Accuracy of the Video_odenet network on the test set: 83.59 %
#Time: 1182.02 seconds

#Test 5   7 Epochs 120/60
#Accuracy of the Video_odenet network on the test set: 95.27 %
#Time: 960.08 seconds

#Test 6   7 Epochs 120/60
#Accuracy of the Video_odenet network on the test set: 96.17 %
#Time: 1041.72 seconds

#Test 7   7 Epochs 120/60
#Accuracy of the Video_odenet network on the test set: 95.07 %
#Time: 987.66 seconds

#Test 8   7 Epochs 120/60
#Accuracy of the Video_odenet network on the test set: 95.67 %
#Time: 955.01 seconds

Accuracy of the Video_odenet network on the test set: 95.27 %
Time: 960.08 seconds


95.26686807653574